In [ ]:
import gymnasium as gym
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.
import Sequential
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

# APPLYING DQN FROM stable_baseline3 LIBRARY :

In [ ]:
"""
from stable_baselines3 import DQN
env = gym.make("LunarLander-v3", render_mode="rgb_array")
observation, info = env.reset()

episode_over = False
while not episode_over:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    episode_over = terminated or truncated

env.close()"""

In [ ]:
"""import numpy as np
env = gym.make("LunarLander-v3", render_mode="rgb_array")
# Initialize DQN agent
model = DQN('MlpPolicy', env, learning_rate=3e-4, verbose=1)

# Train the model
model.learn(total_timesteps=300000)
"""

In [ ]:
"""for i in range(20) :
    env = gym.make("LunarLander-v3", render_mode="human")
    obs, info = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs)
        action = int(action.item())
        obs, reward, done, truncated, info = env.step(action)
        env.render()
    print(reward)"""

# APPLYING DQN FROM SCRATCH :

In [20]:
env = gym.make('LunarLander-v3')
state_size = env.observation_space.shape
num_actions = env.action_space.n
print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (8,)
Number of actions: 4


In [21]:
experiences = deque(maxlen=10000)

In [57]:
SEED = 0
E_MIN = 0.01
E_DECAY = 0.995
TAU = 1e-3
ALPHA = 1e-3
random.seed(SEED)
def get_epsilon(epsilon) :
    return max(E_MIN, epsilon*E_DECAY)

In [58]:
def epsilon_greedy_policy(state, epsilon) :
    proba = random.random()
    if proba > epsilon :
        #print(f'action = {max(target_q_network.predict(state.reshape(1, -1)))}' )
        q_values = target_q_network(state.reshape(1,-1))
        return np.argmax(q_values.numpy()[0])
    else : 
        return random.choice(np.arange(4).tolist())

In [ ]:
def soft_update(tau=0.001):
    weights = q_network.get_weights()
    target_weights = target_q_network.get_weights()  # Fix: Use target_q_network, not target_weight
    for i in range(len(weights)):
        target_weights[i] = tau * weights[i] + (1 - tau) * target_weights[i]
    target_q_network.set_weights(target_weights)

TAU = 1e-3
def soft_update_temp():
    for target_weights, q_net_weights in zip(
        target_q_network.weights, q_network.weights
    ):
        target_weights.assign(TAU * q_net_weights + (1.0 - TAU) * target_weights)


In [60]:
def compute_cost(sampled_experiences, gamma) :
    states ,actions,rewards, next_states,dones = zip(*sampled_experiences)
    states = np.array(states)
    next_states = np.array(next_states)
    rewards = np.array(rewards)
    dones = np.array(dones)
    
    max_qsa = tf.reduce_max(target_q_network(next_states), axis=-1)
    y_targets = rewards + (1-dones)*gamma*max_qsa
    q_values = q_network(states)
    q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]),
                                                tf.cast(actions, tf.int32)], axis=1))
    
    loss = MSE(y_targets,q_values)
    
    return loss

In [67]:
def gd_step1(sampled_experiences, gamma) :
    # compute the loss first
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    with tf.GradientTape() as tape :
        loss = compute_cost(sampled_experiences, gamma)
    gradients = tape.gradient(loss, q_network.trainable_variables)
    optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

def gd_step(experiences, gamma) : 
    """
    Updates the weights of the Q networks.
    
    Args:
      experiences: (tuple) tuple of ["state", "action", "reward", "next_state", "done"] namedtuples
      gamma: (float) The discount factor.
    
    """
    
    # Calculate the loss
    with tf.GradientTape() as tape:
        loss = compute_cost(experiences, gamma)

    # Get the gradients of the loss with respect to the weights.
    gradients = tape.gradient(loss, q_network.trainable_variables)
    
    # Update the weights of the q_network.
    optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

In [70]:
q_network = Sequential([
    Input(shape=state_size),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_actions, activation='linear')
])
target_q_network = Sequential([
    Input(shape=state_size),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_actions, activation='linear')
])

#target_q_network.set_weights(q_network.get_weights())
optimizer = Adam(learning_rate= ALPHA)

In [71]:
env = gym.make('LunarLander-v3')
obs, info = env.reset()
experiences = deque(maxlen=10000)
experiences.append((obs,0,0,obs,False))
def deep_q_network(episodes, gamma) :
    epsilon = 1.0
    t = 0
    for episode in range(episodes) :
        obs, info = env.reset()
        done = False
        while not done :
            current_state = obs
            epsilon = get_epsilon(epsilon)
            action = epsilon_greedy_policy(current_state,epsilon)
            #print(action)
            next_state, reward, done, truncated, info = env.step(action)
            done = done or truncated
            experiences.append((current_state,action,reward, next_state,done))
            obs = next_state
            if (t+1) % 4 == 0 and len(experiences)>64:
                print('##########')
                print(f'episode : {episode}')
                print(f'iteration t = {t}')
                print(f'{len(experiences)} experience ')
                print('##########')
                sampled_experiences = random.sample(experiences, 64)
                gd_step(sampled_experiences, gamma)
                soft_update()
            t+=1
gamma = 0.995
deep_q_network(500, gamma)

##########
episode : 0
iteration t = 63
65 experience 
##########
##########
episode : 0
iteration t = 67
69 experience 
##########
##########
episode : 0
iteration t = 71
73 experience 
##########
##########
episode : 0
iteration t = 75
77 experience 
##########
##########
episode : 0
iteration t = 79
81 experience 
##########
##########
episode : 0
iteration t = 83
85 experience 
##########
##########
episode : 1
iteration t = 87
89 experience 
##########
##########
episode : 1
iteration t = 91
93 experience 
##########
##########
episode : 1
iteration t = 95
97 experience 
##########
##########
episode : 1
iteration t = 99
101 experience 
##########
##########
episode : 1
iteration t = 103
105 experience 
##########
##########
episode : 1
iteration t = 107
109 experience 
##########
##########
episode : 1
iteration t = 111
113 experience 
##########
##########
episode : 1
iteration t = 115
117 experience 
##########
##########
episode : 1
iteration t = 119
121 experience 
##########

KeyboardInterrupt: 

In [ ]:
env = gym.make('LunarLander-v3')
obs, info = env.reset()
done = False
while not done:
    action = np.argmax(target_q_network(obs.reshape(1,-1)).numpy()[0])
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    print(reward)
print(reward)